In [1]:
import numpy as np

In [2]:
with open("input_15.txt", "r") as fh:
    content = fh.read().split("\n\n")
grid = np.array([[i for i in r] for r in content[0].split("\n")])
directions = [d for d in content[1] if d in [">", "<", "v", "^"]]

In [3]:
def pretty_print(grid):
    for r in grid:
        print("".join(r))

In [4]:
# 15a
def move_robot(grid, direction):
    grid_output = grid.copy()
    robot_loc = tuple(rl.item() for rl in np.where(grid == "@"))
    if direction == "<":
        first_wall = np.where(grid[robot_loc[0], :robot_loc[1]] == "#")[0][-1]
        empty_spaces = np.where(grid[robot_loc[0], :robot_loc[1]] == ".")[0]
        if len(empty_spaces) and empty_spaces[-1] > first_wall:
            empty_space = empty_spaces[-1]
            move_block = grid[robot_loc[0], empty_space+1:robot_loc[1]+1]
            grid_output[robot_loc[0], empty_space:robot_loc[1]] = move_block
            grid_output[robot_loc] = "."
    elif direction == ">":
        first_wall = np.where(grid[robot_loc[0], robot_loc[1]+1:] == "#")[0][0] + robot_loc[1] + 1
        empty_spaces = np.where(grid[robot_loc[0], robot_loc[1]+1:] == ".")[0]
        if len(empty_spaces) and empty_spaces[0] + robot_loc[1] + 1 < first_wall:
            empty_space = empty_spaces[0] + robot_loc[1] + 1
            move_block = grid[robot_loc[0], robot_loc[1]:empty_space]
            grid_output[robot_loc[0], robot_loc[1]+1:empty_space+1] = move_block
            grid_output[robot_loc] = "."
    elif direction == "^":
        first_wall = np.where(grid[:robot_loc[0], robot_loc[1]] == "#")[0][-1]
        empty_spaces = np.where(grid[:robot_loc[0], robot_loc[1]] == ".")[0]
        if len(empty_spaces) and empty_spaces[-1] > first_wall:
            empty_space = empty_spaces[-1]
            move_block = grid[empty_space+1:robot_loc[0]+1, robot_loc[1]]
            grid_output[empty_space:robot_loc[0], robot_loc[1]] = move_block
            grid_output[robot_loc] = "."
    elif direction == "v":
        first_wall = np.where(grid[robot_loc[0]+1:, robot_loc[1]] == "#")[0][0] + robot_loc[0] + 1
        empty_spaces = np.where(grid[robot_loc[0]+1:, robot_loc[1]] == ".")[0]
        if len(empty_spaces) and empty_spaces[0] + robot_loc[0] + 1 < first_wall:
            empty_space = empty_spaces[0] + robot_loc[0] + 1
            move_block = grid[robot_loc[0]:empty_space, robot_loc[1]]
            grid_output[robot_loc[0]+1:empty_space+1, robot_loc[1]] = move_block
            grid_output[robot_loc] = "."
    return grid_output


def calculate_gps_boxes(grid):
    boxes = np.where(grid == "O")
    return sum(100*boxes[0]) + sum(boxes[1])


grid_15a = grid.copy()
for d in directions:
    grid_15a = move_robot(grid_15a, d)

print("15a:", calculate_gps_boxes(grid_15a))

15a: 1517819


In [5]:
# 15b
def expand_grid(grid):
    grid_out = np.full((grid.shape[0], grid.shape[1]*2), ".", dtype=str)
    for ri in range(grid.shape[0]):
        for ci in range(grid.shape[1]):
            if grid[ri, ci] == "O":
                grid_out[ri, 2*ci] = "["
                grid_out[ri, 2*ci+1] = "]"
            else:
                grid_out[ri, 2*ci] = grid[ri, ci]
                if grid[ri, ci] != "@":
                    grid_out[ri, 2*ci+1] = grid[ri, ci]
    return grid_out


def move_robot_15b(grid, direction):
    # Left and right movements don't change
    if direction == "<" or direction == ">":
        return move_robot(grid, direction)

    # Up and down movements are more complicated
    grid_out = grid.copy()
    robot_loc = tuple(rl.item() for rl in np.where(grid == "@"))
    if direction == "^":
        if grid_out[robot_loc[0]-1, robot_loc[1]] == ".":
            # No boxes to move
            grid_out[robot_loc[0]-1, robot_loc[1]] = "@"
            grid_out[robot_loc] = "."
            return grid_out
        elif grid_out[robot_loc[0]-1, robot_loc[1]] == "#":
            # Blocked
            return grid_out
        else:
            search_height = robot_loc[0]-1
            if grid_out[search_height, robot_loc[1]] == "[":
                boxes_to_move = [set([(search_height, robot_loc[1]), (search_height, robot_loc[1]+1)])]
            else:
                boxes_to_move = [set([(search_height, robot_loc[1]-1), (search_height, robot_loc[1])])]
            
            keep_searching = True
            while keep_searching:
                new_boxes_to_move = set()
                keep_searching = False
                search_height -= 1
                for br, bc in boxes_to_move[-1]:
                    if grid_out[search_height, bc] == "#":
                        return grid_out
                    if grid_out[search_height, bc] == "[":
                        keep_searching = True
                        new_boxes_to_move.add((search_height, bc))
                        new_boxes_to_move.add((search_height, bc+1))
                    elif grid_out[search_height, bc] == "]":
                        keep_searching = True
                        new_boxes_to_move.add((search_height, bc-1))
                        new_boxes_to_move.add((search_height, bc))
                boxes_to_move.append(new_boxes_to_move)
            # Move boxes
            for layer in boxes_to_move[::-1]:
                for br, bc in layer:
                    grid_out[br-1, bc] = grid_out[br, bc]
                    grid_out[br, bc] = "."
            grid_out[robot_loc[0]-1, robot_loc[1]] = "@"
            grid_out[robot_loc] = "."
            return grid_out
            
    elif direction == "v":
        if grid_out[robot_loc[0]+1, robot_loc[1]] == ".":
            # No boxes to move
            grid_out[robot_loc[0]+1, robot_loc[1]] = "@"
            grid_out[robot_loc] = "."
            return grid_out
        elif grid_out[robot_loc[0]+1, robot_loc[1]] == "#":
            # Blocked
            return grid_out
        else:
            search_height = robot_loc[0]+1
            if grid_out[search_height, robot_loc[1]] == "[":
                boxes_to_move = [set([(search_height, robot_loc[1]), (search_height, robot_loc[1]+1)])]
            else:
                boxes_to_move = [set([(search_height, robot_loc[1]-1), (search_height, robot_loc[1])])]
            
            keep_searching = True
            while keep_searching:
                new_boxes_to_move = set()
                keep_searching = False
                search_height += 1
                for br, bc in boxes_to_move[-1]:
                    if grid_out[search_height, bc] == "#":
                        return grid_out
                    if grid_out[search_height, bc] == "[":
                        keep_searching = True
                        new_boxes_to_move.add((search_height, bc))
                        new_boxes_to_move.add((search_height, bc+1))
                    elif grid_out[search_height, bc] == "]":
                        keep_searching = True
                        new_boxes_to_move.add((search_height, bc-1))
                        new_boxes_to_move.add((search_height, bc))
                boxes_to_move.append(new_boxes_to_move)
            # Move boxes
            for layer in boxes_to_move[::-1]:
                for br, bc in layer:
                    grid_out[br+1, bc] = grid_out[br, bc]
                    grid_out[br, bc] = "."
            grid_out[robot_loc[0]+1, robot_loc[1]] = "@"
            grid_out[robot_loc] = "."
            return grid_out

def calculate_gps_boxes(grid):
    boxes = np.where(grid == "[")
    return sum(100*boxes[0]) + sum(boxes[1])
    

grid_15b = expand_grid(grid)
for d in directions:
    grid_15b = move_robot_15b(grid_15b, d)

print("15b:", calculate_gps_boxes(grid_15b))

15b: 1538862
